In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [73]:
df = pd.read_csv('../data/df_processed.csv', index_col=0)
df = df.reset_index()
df = df.dropna()
df.head()

,tx_amount,tx_fraud,mean_amount,std_amount,mean_nb_tx_per_day,nb_terminals,amount_above_mean,deviation_std,deviation_big,hour,day_of_week,log_amount,minutes_since_last_tx,distance_km
0,61.51,0,62.262521,31.13126,2.179533,22,-0.752521,-0.02417,0,3,6,4.11920,150.0,200.59
1,129.61,0,62.262521,31.13126,2.179533,22,67.347479,2.16334,1,5,6,4.86453,150.0,459.27
2,96.50,0,62.262521,31.13126,2.179533,22,34.237479,1.09978,0,10,6,4.56954,310.0,384.02
3,82.19,0,62.262521,31.13126,2.179533,22,19.927479,0.64011,0,15,6,4.40903,298.0,286.74
4,50.77,0,62.262521,31.13126,2.179533,22,-11.492521,-0.36916,0,19,6,3.92731,227.0,73.28


In [74]:
df['tx_fraud'].isna().sum()

0

In [75]:
df.isna().sum()

tx_amount                0
tx_fraud                 0
mean_amount              0
std_amount               0
mean_nb_tx_per_day       0
nb_terminals             0
amount_above_mean        0
deviation_std            0
deviation_big            0
hour                     0
day_of_week              0
log_amount               0
minutes_since_last_tx    0
distance_km              0
dtype: int64

In [76]:
X = df.drop(['tx_fraud'], axis=1)
y = df['tx_fraud']

In [77]:
X.dtypes

tx_amount                float64
mean_amount              float64
std_amount               float64
mean_nb_tx_per_day       float64
nb_terminals               int64
amount_above_mean        float64
deviation_std            float64
deviation_big              int64
hour                       int64
day_of_week                int64
log_amount               float64
minutes_since_last_tx    float64
distance_km              float64
dtype: object

In [78]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

print("Number of rows in X_train:", len(X_train))
print("Number of rows in X_test:", len(X_test))
print("Number of rows in y_train:", len(y_train))
print("Number of rows in y_test:", len(y_test))
X_train.shape

Number of rows in X_train: 232984
Number of rows in X_test: 58247
Number of rows in y_train: 232984
Number of rows in y_test: 58247


(232984, 13)

In [81]:
class Normalizer:
    def __init__(self):
        self.mean = None
        self.std = None

    def fit(self, X):
        self.mean = np.mean(X, axis=0)
        self.std = np.std(X, axis=0)

    def transform(self, X):
        X_normalized = (X - self.mean) / self.std
        return X_normalized

    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)



In [82]:

from sklearn.model_selection import cross_val_score

scaler = Normalizer()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
lr = LogisticRegression()

scores = cross_val_score(estimator=lr, X=X_train_scaled, y=y_train, cv=5)
print(f"Cross-validation scores: {scores}")
print(f"Mean cross-validation score: {scores.mean():.3f}")


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1196, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\PC\AppData\Local\Programs\Pyt

In [ ]:
# Print the cross-validation scores
print("Cross-Validation Scores:", scores)
print("Average Cross-Validation Score:", scores.mean())

In [ ]:
np.any(np.isnan(X))

False